In [20]:
import requests, pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
from dotenv import load_dotenv
import os
import schedule
import time
from datetime import datetime

In [21]:
load_dotenv()  # This automatically finds and loads the .env file


True

In [22]:


# Extract
api_key = os.getenv("API_KEY")

url = "https://api.currencybeacon.com/v1/latest"

params = {
    "base": "IDR",
    "api_key": api_key
}

response = requests.get(url, params=params)

# Convert to JSON
data = response.json()

# Print the result
print(data)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print("Error:", response.status_code, response.text)






{'meta': {'code': 200, 'disclaimer': 'Usage subject to terms: https://currencybeacon.com/terms'}, 'response': {'date': '2025-11-02T05:22:07Z', 'base': 'IDR', 'rates': {'ZWL': 3.94953894, 'ADA': 9.796e-05, 'AED': 0.00022074, 'AFN': 0.00400031, 'ALL': 0.00503936, 'AMD': 0.02330163, 'ANG': 0.00010801, 'AOA': 0.05484542, 'ARS': 0.08671764, 'ATS': 0.0007133, 'AUD': 9.189e-05, 'AWG': 0.00010759, 'AZM': 0.51120146, 'AZN': 0.00010224, 'BAM': 0.00010139, 'BBD': 0.00012021, 'BCH': 1.1e-07, 'BDT': 0.00734829, 'BEF': 0.00209113, 'BGN': 0.00010139, 'BHD': 2.26e-05, 'BIF': 0.17744245, 'BMD': 6.011e-05, 'BND': 7.814e-05, 'BOB': 0.00041634, 'BRL': 0.00032326, 'BSD': 6.011e-05, 'BTC': 0, 'BTN': 0.0053434, 'BWP': 0.00085937, 'BYN': 0.00020464, 'BYR': 2.04638604, 'BZD': 0.00012084, 'CAD': 8.432e-05, 'CDF': 0.1353113, 'CHF': 4.842e-05, 'CLF': 1.44e-06, 'CLP': 0.05665316, 'CNH': 0.00042761, 'CNY': 0.00042757, 'COP': 0.23187523, 'CRC': 0.03014753, 'CUC': 6.011e-05, 'CUP': 0.00144307, 'CVE': 0.00571615, 'CYP

In [23]:
response_data = data["response"]
rates = response_data["rates"]
base = response_data["base"]
timestamp = response_data["date"]

# Convert to DataFrame
df = pd.DataFrame(list(rates.items()), columns=["Currency", "Rate"])

# Add extra columns
df["Base"] = base
df["Last_Updated"] = timestamp

# Optional: sort and round
df["Rate"] = df["Rate"].round(6)
df = df.sort_values(by="Rate", ascending=False)
df = df.reset_index(drop=True)

df.head()

,Currency,Rate,Base,Last_Updated
0,TRL,2526.967262,IDR,2025-11-02T05:22:07Z
1,VEF,1341.217195,IDR,2025-11-02T05:22:07Z
2,GHC,6.542580,IDR,2025-11-02T05:22:07Z
3,LBP,5.387806,IDR,2025-11-02T05:22:07Z
4,ZWL,3.949539,IDR,2025-11-02T05:22:07Z


In [24]:
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [25]:
engine = create_engine(connection_string)

df.to_sql("currency_rates", engine, if_exists="replace", index=False)

print("Data successfully loaded into PostgreSQL!")

Data successfully loaded into PostgreSQL!


In [26]:
# schedule.every().day.at("18:00").do(etl_currency_rates)

# print("Scheduler started. ETL will run daily at 18:00...")

# # Run once immediately if you want
# etl_currency_rates()

# while True:
#     schedule.run_pending()
#     time.sleep(60)  # Check every minute